In [ ]:
from toolbox.store.callbacks.auth.auth_slack_keyring import get_tokens_and_cookie, get_cookie, get_tokens, get_config


In [ ]:
import json

In [ ]:

def remove_control_chars(s):
    """Remove ASCII control characters (0–31) except \n, \r, \t."""
    s = s.replace('\x00', '')

    return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", s)

In [ ]:
def fix_quotes_heuristic(s):
    """Fix malformed quotes like '000"".""0' → '000.0'."""
    # Replace duplicated quotes between numbers/words
    s = re.sub(r'"([^"]*?)"\s*"\s*\.\s*"\s*"([^"]*?)"', r'"\1.\2"', s)
    return s

In [ ]:
import leveldb
from toolbox.store.callbacks.auth.auth_slack_keyring import get_config, SLACK_LEVELDB_PATH, try_to_copy_and_read_leveldb, remove_control_chars
import json

db = try_to_copy_and_read_leveldb(SLACK_LEVELDB_PATH)
cfg = next(v for k, v in db.RangeIter() if bytearray(b"localConfig_v2") in k)
decoded_cfg = cfg[1:].decode("utf-8", errors="ignore")
cleaned_cfg = remove_control_chars(decoded_cfg)
cleaned_cfg = fix_quotes_heuristic(cleaned_cfg)
try:
    cfg = json.loads(cleaned_cfg)
except Exception as e:
    cleaned_cfg = cleaned_cfg + "}"
    cfg = json.loads(cleaned_cfg)
    

In [ ]:
fix_quotes_heuristic('"startTs": "000"".""0"')

In [ ]:
cleaned_cfg

In [ ]:
import json
import re

def remove_control_chars(s):
    """Remove ASCII control characters (0–31) except \n, \r, \t."""
    s = s.replace('\x00', '')

    return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", s)

def fix_quotes_heuristic(s):
    """Fix malformed quotes like '000"".""0' → '000.0'."""
    # Replace duplicated quotes between numbers/words
    s = re.sub(r'(\d)""(\.?)""(\d)', r'\1\2\3', s)
    s = re.sub(r'""', '"', s)
    return s

def try_json_repair(s):
    """Try to repair malformed JSON by applying heuristics."""
    try:
        s = fix_quotes_heuristic(s)
        return json.loads(s)
    except json.JSONDecodeError:
        

        # Common fix: if string ends without a closing brace
        s += "}"

        try:
            return json.loads(s)
        except json.JSONDecodeError as e:
            raise RuntimeError("Failed to parse Slack config JSON.") from e


    try:
        raw = cfg.decode('utf-8', errors='replace')  # use 'replace' to avoid decode crashes
    except UnicodeDecodeError as e:
        raise RuntimeError("Failed to decode Slack config bytes as UTF-8.") from e

    cleaned = remove_control_chars(raw)

    return try_json_repair(cleaned)

In [ ]:
get_config(db)

In [ ]:
import json
import re

def remove_control_chars(s):
    # Remove all non-printable control characters (ASCII 0–31), except \n, \r, \t
    return re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F]", "", s)

def safe_json_load(s):
    """
    Attempts to safely load malformed JSON by fixing common issues.
    """
    try:
        return json.loads(s)
    except json.JSONDecodeError as e:
        # Try to handle common issue: extra unescaped quotes
        fixed = s.replace('""', '"').replace('"""', '"')
        try:
            return json.loads(fixed)
        except Exception:
            # Try trimming or adding closing brace if missing
            if not fixed.strip().endswith("}"):
                fixed += "}"
            try:
                return json.loads(fixed)
            except Exception:
                raise RuntimeError("JSON still malformed after fixes.") from e


        # Find the key with 'localConfig_v2' in it
cfg = next(v for k, v in db.RangeIter() if b"localConfig_v2" in k)
try:
    # Slack sometimes prefixes with a status byte, skip it if needed
    if cfg.startswith(b"{"):
        raw_json = cfg.decode("utf-8")
    else:
        raw_json = cfg[1:].decode("utf-8")

    cleaned_json = remove_control_chars(raw_json)

    # Attempt safe load
    res = safe_json_load(cleaned_json)

except Exception as e:
    raise RuntimeError("Slack config is malformed or unreadable.") from e


In [ ]:
cfg

In [ ]:
xoxd_cookie = get_cookie()

In [ ]:
# xoxd_cookie = get_slack_xoxd_cookie()
from slack_sdk import WebClient
import os

slack_token = os.getenv("SLACK_TOKEN")  # Your API token (xoxc-...)

import requests

cookies = {"d": xoxd_cookie}
headers = {"User-Agent": "Mozilla/5.0"}
slack_token = os.getenv("SLACK_TOKEN")

response = requests.post(
    "https://slack.com/api/auth.test",
    headers=headers,
    cookies=cookies,
    data={"token": slack_token},
)
response_json = response.json()
assert response_json["ok"] == True

# headers = {"Cookie": f"d={xoxd_cookie}", "User-Agent": "Mozilla/5.0"}
# client = WebClient(token=slack_token, headers=headers)

# response = client.auth_test()
# assert response.data["ok"] == True

In [ ]:
tokens_and_cookie = get_tokens_and_cookie()

In [ ]:
token = tokens_and_cookie['tokens']["https://openmined.slack.com/"]["token"]

old: 'xoxc-213207348208etcetcetc'


In [ ]:
import sqlite3

db_path = "/Users/koen/Library/Containers/com.tinyspeck.slackmacgap/Data/Library/Application Support/Slack/Cookies"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Check if there's a meta table with version info
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# Try to get version info
try:
    cursor.execute("SELECT * FROM meta;")
    print("Meta table:", cursor.fetchall())
except:
    pass

conn.close()

In [ ]:
cookie

In [ ]:
from slack_sdk import WebClient
headers = {"Cookie": f"d={cookie}", "User-Agent": "Mozilla/5.0 (compatible; Python)"}
client = WebClient(token=token, headers=headers)

In [ ]:
res = client.auth_test()

In [ ]:

res.data